In [10]:
import numpy as np
import pandas as pd

In [11]:
# u.user 파일을 DataFrame으로 읽기

# DataFrame의 열 이름을 저장하는 리스트
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

''' 해당 경로의 CSV 파일 읽어와서 DataFrame으로 변환함
    names=u : 열 이름을 u_cols 리스트에 정의된 열 이름으로 설정함
'''
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1')

# user_id 열을 인덱스로 설정함. 
users = users.set_index('user_id')

# DataFrame의 첫 5개 행을 출력하여 확인함
users

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [12]:
# u.item 파일을 DataFrame으로 읽기
# 영화 정보
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()


,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [13]:
# u.data 파일을 DataFrame으로 읽기
# 영화 펑점 정보
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/u.data', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.set_index('user_id')
ratings.head()


,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [14]:
# 인기 제품 방식
# Best-seller 추천을 위한 함수

'''
n_items : 추천할 영화의 갯수
movie_sort : 각 영화의 평점 평균 movie_mean을 내림차순으로 정렬,
            상위부터 지정된 갯수(n_items)만큼 저장함.
            이때 movie_mean이 Pandas Series이므로 movie_sort도 같은 형식
            즉, 각각의 값(평점 평균)과 index가 같이 저장됨
recom_movies : 정렬된 영화 중에서 상위 n개의 영화 정보
recommendatiosn : 선정된 상위 n개의 영화 제목
'''
def recom_movie1(n_items=5):
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

# 평균 평점 계산
# ratings에 있는 평가 데이터를 groupby()로 각 영화별 평점의 평균을 구함
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()

# Best-seller 추천 호출 : 상위 5개의 영화를 추천하기 위해 영화 제목 출력함.
recom_movie1(5)


movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

In [15]:
# 위의 recom_movie1과 똑같은 일을 하는데 한줄로 만든 것
def recom_movie2(n_items):
   return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

In [16]:

# 정확도 계산 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

rmse = []
for user in set(ratings.index):
    y_true = ratings.loc[user]['rating']
    y_pred = movie_mean[ratings.loc[user]['movie_id']]
    accuracy = RMSE(y_true, y_pred)
    rmse.append(accuracy)
print(np.mean(rmse))


0.996007224010567
